In [1]:
!pip install transformers accelerate
!pip install transformers[sentencepiece]
!pip install sentencepiece
from transformers import AutoTokenizer, AutoModel, EarlyStoppingCallback, AutoModelForSequenceClassification, AutoConfig,Trainer, TrainingArguments,DataCollatorWithPadding
import torch
import numpy as np
import pandas as pd
!pip install datasets
from datasets import load_metric
%pip install evaluate
from evaluate import evaluator
from sklearn.model_selection import train_test_split
import csv
!pip install openprompt
import os
import csv
from openprompt.data_utils import InputExample

from datasets import Dataset, load_metric
from sklearn import preprocessing
# notebook_login()
from tqdm import tqdm
CUDA_LAUNCH_BLOCKING=1


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 492.4/492.4 kB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 15.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 24.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 16.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 kB 2.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 146.4/146.4 kB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 50.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.6/101.6 kB 14.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 304.5/304.5 kB 30.0 MB/s eta 0:00:00
  Attempting uninstall: sentencepiece
    Found existing installation: sentencepiece 0.1.99
    Uninstalling sentencepiece-0.1.99:
      Successfully uninstalled sentencepiece-0.1.99
  Attempting uninstall: protobuf
    Found existing installation: protobuf 3.20.3
    Unins

/usr/local/lib/python3.10/dist-packages/transformers/generation_utils.py:24: FutureWarning: Importing `GenerationMixin` from `src/transformers/generation_utils.py` is deprecated and will be removed in Transformers v5. Import as `from transformers import GenerationMixin` instead.
  warnings.warn(


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
train_I = pd.read_csv(r'/content/drive/MyDrive/dataset I/train.csv', encoding='utf_8_sig')
train_I.fillna('', inplace=True)
train_II = pd.read_csv(r'/content/drive/MyDrive/dataset II/train.csv', encoding='utf_8_sig')
train_II.fillna('', inplace=True)
train = pd.concat([train_I,train_II],axis=0)
test_I = pd.read_csv(r'/content/drive/MyDrive/dataset II/test.csv', encoding='utf_8_sig')
test_I.fillna('', inplace=True)
test_II = pd.read_csv(r'/content/drive/MyDrive/dataset II/test.csv', encoding='utf_8_sig')
test_II.fillna('', inplace=True)

In [4]:
# unified columns
train = train[['label','text']].reset_index(drop=True)
label2id = {'positive':0,'negative':1,'Adaptive':2,'Perfective':3,'Corrective':4}
train = train.replace({'label':label2id})
test_I = test_I.replace({'label':label2id})
test_II = test_II.replace({'label':label2id})

In [5]:
len(train)

8333

In [6]:
train

,label,text
0,0,XSS 취약점 수정
1,1,"Auto merge of #4754 - daira:fix-enable-debug, ..."
2,0,CAMEL-9309: Make it easier to turn on|off java...
3,0,Fixed bug #72627: Memory Leakage In exif_proce...
4,1,Merge branch 'dev'
...,...,...
8328,2,glib-2.0: add GLib.HashTable.foreach\n\nFixes ...
8329,2,"search is more complete, support forward/rever..."
8330,4,"Extract Function: Consider reference ""broken"" ..."
8331,3,First part of https://jira.duraspace.org/brows...


In [8]:
test_I

,Unnamed: 0,commit_id,project,text,label,ADDING_ATTRIBUTE_MODIFIABILITY,ADDING_METHOD_OVERRIDABILITY,ADDITIONAL_CLASS,ADDITIONAL_FUNCTIONALITY,ADDITIONAL_OBJECT_STATE,...,Extract Variable,Inline Variable,Parametrize Variable,Rename Variable,Replace Variable with Attribute,Change Variable Type,Move and Rename Class,Move Class,Extract and Move Method,Move Source Folder
0,236,d1bdfa2c80fa011aef572a598e7a1e0671f70e25,intellij-community,cleanup--,3,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1430,d008929c8439edf66bd9bb428201217304eacfea,Vala,libxml-2.0: Fix instance position in several m...,4,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1039,f5a3477fe1bd136a3f9f7e2048299ad24bd47ded,Valadoc,gtkdoc-importer: Add support for computeroutput\n,2,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,535,7be3d247f90c23bbb11bc735276d51008243a7f6,kotlin,fix tests after recent refactoring--,4,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1135,a449a6993aa2850d828e87676210224b6f24d35b,tapiji,updated licenses\n,3,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
533,746,6945d04785d7322df45b5c71053ba18431f606cd,orientdb,Further work or traverse: - supported new- OSQ...,2,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
534,47,d43b3d1f76bc13cd4c2bf8cb4f951f751aea86e6,ReactiveX-RxJava,Fix autoConnect calling onStart twice.--,4,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
535,424,8c475876dda2507977fd7282c37462136400daf2,drools,-Added fixes for waltz to run waltz50--git-svn...,4,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
536,660,ebfb482c3ee955f24be4d2802c1e06d6f0aa0d1c,restlet-framework-java,Removed debug traces.--,3,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [9]:
from openprompt.data_utils import InputExample

dataset = {}
# for split in ['train','test']:
    # dataset[split] = []
dataset['train'] = []
dataset['test'] = []
for index, row in train.iterrows():
# print(row.text)
# print(row['text'])
  input_example = InputExample(text_a  =  row['text'], label=int(row['label']))
  dataset['train'].append(input_example)
for index, row in test_II.iterrows():
    # print(row.text)
    # print(row['text'])
    input_example = InputExample(text_a  =  row['text'], label=int(row['label']))
    dataset['test'].append(input_example)
print(dataset['train'][0])

{
  "guid": null,
  "label": 0,
  "meta": {},
  "text_a": "XSS \ucde8\uc57d\uc810 \uc218\uc815",
  "text_b": "",
  "tgt_text": null
}



In [10]:
len(train)

8333

In [11]:
len(test_II)

538

In [12]:
from openprompt.plms import load_plm

plm, tokenizer, model_config, WrapperClass = load_plm("t5", "google/flan-t5-base")


You are using the legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This means that tokens that come after special tokens will not be properly handled. We recommend you to read the related pull request available at https://github.com/huggingface/transformers/pull/24565


In [13]:
# Constructing Template
# A template can be constructed from the yaml config, but it can also be constructed by directly passing arguments.
from openprompt.prompts import ManualTemplate
template_text = 'This {"placeholder":"text_a"} is a {"mask"} commit.'
mytemplate = ManualTemplate(tokenizer=tokenizer, text=template_text)

# To better understand how does the template wrap the example, we visualize one instance.

wrapped_example = mytemplate.wrap_one_example(dataset['train'][0])
print(wrapped_example)

[[{'text': 'This', 'loss_ids': 0, 'shortenable_ids': 0}, {'text': ' XSS 취약점 수정', 'loss_ids': 0, 'shortenable_ids': 1}, {'text': ' is a', 'loss_ids': 0, 'shortenable_ids': 0}, {'text': '<mask>', 'loss_ids': 1, 'shortenable_ids': 0}, {'text': ' commit.', 'loss_ids': 0, 'shortenable_ids': 0}], {'label': 0}]


In [14]:
wrapped_t5tokenizer = WrapperClass(max_seq_length=128, decoder_max_length=3, tokenizer=tokenizer,truncate_method="head")
# or
from openprompt.plms import T5TokenizerWrapper
wrapped_t5tokenizer= T5TokenizerWrapper(max_seq_length=128, decoder_max_length=3, tokenizer=tokenizer,truncate_method="head")

# You can see what a tokenized example looks like by
tokenized_example = wrapped_t5tokenizer.tokenize_one_example(wrapped_example, teacher_forcing=False)
print(tokenized_example)
print(tokenizer.convert_ids_to_tokens(tokenized_example['input_ids']))
print(tokenizer.convert_ids_to_tokens(tokenized_example['decoder_input_ids']))


{'input_ids': [100, 3, 4, 4256, 3, 2, 3, 2, 19, 3, 9, 32099, 10042, 5, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'decoder_input_ids': [0, 32099, 0], 'loss_ids': [0, 1, 0]}
['▁This', '▁', 'X', 'SS', '▁', '<unk>', '▁', '<unk>', '▁is', '▁', 'a', '<extra_id_0>', '▁commit', '.', '</s>', '<pad>', '<p

In [15]:
model_inputs = {}
for split in ['train', 'test']:
    model_inputs[split] = []
    for sample in dataset[split]:
        tokenized_example = wrapped_t5tokenizer.tokenize_one_example(mytemplate.wrap_one_example(sample), teacher_forcing=False)
        model_inputs[split].append(tokenized_example)

Token indices sequence length is longer than the specified maximum sequence length for this model (513 > 512). Running this sequence through the model will result in indexing errors


In [16]:
# We provide a `PromptDataLoader` class to help you do all the above matters and wrap them into an `torch.DataLoader` style iterator.
from openprompt import PromptDataLoader

train_dataloader = PromptDataLoader(dataset=dataset["train"], template=mytemplate, tokenizer=tokenizer,
    tokenizer_wrapper_class=WrapperClass, max_seq_length=256, decoder_max_length=3,
    batch_size=4,shuffle=True, teacher_forcing=False, predict_eos_token=False,
    truncate_method="head")
# next(iter(train_dataloader))

tokenizing: 8333it [00:11, 711.45it/s]


In [17]:
from openprompt.prompts import ManualVerbalizer,KnowledgeableVerbalizer
import torch

# for example the verbalizer contains multiple label words in each class
# myverbalizer = ManualVerbalizer(tokenizer, num_classes=3,
#                         label_words=[["Adaptive"], ["Perfective"], ["Corrective"]])
myverbalizer = KnowledgeableVerbalizer(tokenizer, num_classes=5).from_file("/content/drive/MyDrive/knowledgeable_verbalizer_allset.txt")

print(myverbalizer.label_words_ids)
logits = torch.randn(2,len(tokenizer)) # creating a pseudo output from the plm, and
print(myverbalizer.process_logits(logits)) # see what the verbalizer do


##Num of label words for each label: [21, 20, 19, 20, 20]
Parameter containing:
tensor([[[ 2451,     0,     0,     0,     0],
         [ 3614,     0,     0,     0,     0],
         [ 1346,     0,     0,     0,     0],
         [  766,     0,     0,     0,     0],
         [ 7992,     0,     0,     0,     0],
         [ 1006,    35,     0,     0,     0],
         [    3, 11005,   524,     0,     0],
         [   16,  4334,     0,     0,     0],
         [    3, 21477,     0,     0,     0],
         [   36,  5595,     0,     0,     0],
         [   16, 12388,   687,   179,     0],
         [    3, 23094,     0,     0,     0],
         [  417,     0,     0,     0,     0],
         [ 3442,     0,     0,     0,     0],
         [  143,     0,     0,     0,     0],
         [ 2210,     0,     0,     0,     0],
         [ 1669,     0,     0,     0,     0],
         [11892,    15,     0,     0,     0],
         [   21,    17,  4921,     0,     0],
         [ 4095,     0,     0,     0,     0],


In [18]:
from openprompt import PromptForClassification

use_cuda = True
prompt_model = PromptForClassification(plm=plm,template=mytemplate, verbalizer=myverbalizer, freeze_plm=False)
if use_cuda:
    prompt_model=  prompt_model.cuda()


In [19]:
# Now the training is standard
from transformers import  AdamW, get_linear_schedule_with_warmup
loss_func = torch.nn.CrossEntropyLoss()
no_decay = ['bias', 'LayerNorm.weight']
# it's always good practice to set no decay to biase and LayerNorm parameters
optimizer_grouped_parameters = [
    {'params': [p for n, p in prompt_model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in prompt_model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]

optimizer = AdamW(optimizer_grouped_parameters, lr=1e-4)



/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [20]:
# Set up early stopping parameters
best_validation_loss = float('inf')
patience = 3  # Number of epochs to wait for improvement
waited_epochs = 0
# Evaluate
validation_dataloader = PromptDataLoader(dataset=dataset["test"], template=mytemplate, tokenizer=tokenizer,
    tokenizer_wrapper_class=WrapperClass, max_seq_length=256, decoder_max_length=3,
    batch_size=4,shuffle=False, teacher_forcing=False, predict_eos_token=False,
    truncate_method="head")

tokenizing: 538it [00:00, 829.23it/s]


In [21]:
for epoch in range(10):
    tot_loss = 0
    for step, inputs in enumerate(train_dataloader):
        if use_cuda:
            inputs = inputs.cuda()
        logits = prompt_model(inputs)
        labels = inputs['label']
        loss = loss_func(logits, labels)
        loss.backward()
        tot_loss += loss.item()
        optimizer.step()
        optimizer.zero_grad()
        if step % 100 == 1:
            print("Epoch {}, average loss: {}".format(epoch, tot_loss / (step + 1)), flush=True)

    # Validation loop
    validation_loss = 0
    prompt_model.eval()
    with torch.no_grad():
        for val_step, val_inputs in enumerate(validation_dataloader):
            if use_cuda:
                val_inputs = val_inputs.cuda()
            val_logits = prompt_model(val_inputs)
            val_labels = val_inputs['label']
            val_loss = loss_func(val_logits, val_labels)
            validation_loss += val_loss.item()

    validation_loss /= (val_step + 1)
    print("Epoch {}, validation loss: {}".format(epoch, validation_loss), flush=True)

    # Check for early stopping
    if validation_loss < best_validation_loss:
        best_validation_loss = validation_loss
        waited_epochs = 0
        # Save the model's state dictionary to a file
        torch.save(prompt_model.state_dict(), "best_model_state_dict.pt")
    else:
        waited_epochs += 1
        if waited_epochs >= patience:
            print("Early stopping at epoch {}.".format(epoch), flush=True)
            break

    prompt_model.train()  # Set the model back to training mode

Epoch 0, average loss: 1.8703283071517944
Epoch 0, average loss: 1.0413476130103363
Epoch 0, average loss: 0.9854561748112193
Epoch 0, average loss: 0.8679354569769853
Epoch 0, average loss: 0.8133839692512703
Epoch 0, average loss: 0.7753239621693988
Epoch 0, average loss: 0.7293802803239949
Epoch 0, average loss: 0.6994215907247743
Epoch 0, average loss: 0.6767932301447576
Epoch 0, average loss: 0.6561334204597774
Epoch 0, average loss: 0.6395773390780191
Epoch 0, average loss: 0.6279080951850043
Epoch 0, average loss: 0.612334077919023
Epoch 0, average loss: 0.5980335277392201
Epoch 0, average loss: 0.5916520163101993
Epoch 0, average loss: 0.5791863997481743
Epoch 0, average loss: 0.5682856454455832
Epoch 0, average loss: 0.5630429229987993
Epoch 0, average loss: 0.5534056321450083
Epoch 0, average loss: 0.5465693529705066
Epoch 0, average loss: 0.5369833150774345
Epoch 0, validation loss: 0.8847774283891475
Epoch 1, average loss: 0.3361235707998276
Epoch 1, average loss: 0.3388909

In [22]:
from sklearn.metrics import classification_report
allpreds = []
alllabels = []
for step, inputs in enumerate(validation_dataloader):
    if use_cuda:
        inputs = inputs.cuda()
    logits = prompt_model(inputs)
    labels = inputs['label']
    alllabels.extend(labels.cpu().tolist())
    allpreds.extend(torch.argmax(logits, dim=-1).cpu().tolist())

# Generate the classification report
class_names = [f"Class_{i}" for i in range(3)]  # Replace num_classes with the number of classes in your dataset
classification_rep = classification_report(alllabels, allpreds, digits=4)

print("Classification Report:")
print(classification_rep)

Classification Report:
              precision    recall  f1-score   support

           0     0.0000    0.0000    0.0000         0
           1     0.0000    0.0000    0.0000         0
           2     0.8696    0.7910    0.8284       177
           3     0.7978    0.8111    0.8044       180
           4     0.8256    0.7845    0.8045       181

    accuracy                         0.7955       538
   macro avg     0.4986    0.4773    0.4875       538
weighted avg     0.8308    0.7955    0.8123       538



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
